# Proyecto 1

___
El reto que se propone en este proyecto consiste en construir un modelo para el problema de clasificación y localización de objetos en imágenes. Para entrenar, validar y probar el modelo, utilizaremos el conjunto de datos Tiny Imagenet que tiene 200 clases. Cada clase tiene 500 imágenes de entrenamiento, 50 imágenes de validación y 50 imágenes de prueba.

El alcance esperado para el primer avance es:
- Datos reorganizados (en caso de ser necesario)
- Pipeline de datos de entrenamiento, validación y pruebas
    - Generadores de datos
    - Aumento de datos (en caso de ser necesario)

In [1]:
# Librerías
import numpy as np
import pandas as pd
import tensorflow as tf

# Archivo .py con funciones
import Utilidades

## 1. Carga de Datos

In [2]:
# Tabla wnids (IDS de las clases que tenemos)
wnids = pd.read_fwf('./Data/wnids.txt', header=None)
wnids.columns = ['Clase'] # Cambiando el nombre a la columna

print(f'Contamos con {wnids["Clase"].nunique()} clases distintas.')
wnids.head()

Contamos con 200 clases distintas.


,Clase
0,n02124075
1,n04067472
2,n04540053
3,n04099969
4,n07749582


In [3]:
# Tabla words (Nombres de las clases)
words = pd.read_fwf('./Data/words.txt', header=None)
words.columns = ['Clase', 'Tipo_Clase', 'NaN']
words = words[['Clase', 'Tipo_Clase']]

# Contando cuántos 'TiposClase' hay por 'Clase'
words = words.groupby('Clase')['Tipo_Clase'].unique().reset_index(drop=False)

# Imprimiendo número de clases que hay en el archivo y el ínicio del dataset
print(f'Contamos con {words["Clase"].nunique()} clases distintas.')
words.head()

Contamos con 82115 clases distintas.


,Clase,Tipo_Clase
0,n00001740,[entity]
1,n00001930,[physical entity]
2,n00002137,"[abstraction, abstract entity]"
3,n00002452,[thing]
4,n00002684,"[object, physical object]"


- Al tener dos archivos distintos, uno con las clases que tenemos "wnids.txt" y otra con las 82115 clases que existen, mergeamos las dos para obtener el 'inner' de los dos conjuntos y obtener una tabla con las clases y la etiqueta.

In [4]:
# Mergeando las tablas por obtener el 'inner' de los dos
clases = words.merge(wnids, on='Clase')
clases

,Clase,Tipo_Clase
0,n01443537,"[goldfish, Carassius auratus]"
1,n01629819,"[European fire salamander, Salamandra salamandr]"
2,n01641577,"[bullfrog, Rana catesbeiana]"
3,n01644900,"[tailed frog, bell toad, ribbed toad, tailed to]"
4,n01698640,"[American alligator, Alligator mississipiensis]"
...,...,...
195,n09246464,"[cliff, drop, drop-off]"
196,n09256479,[coral reef]
197,n09332890,"[lakeside, lakeshore]"
198,n09428293,"[seashore, coast, seacoast, sea-coast]"


## 2. EDA sobre las carpetas de 'Data'

In [5]:
# Path de los datos
path = './Data/'

# Utilizando la función para ver el contenido de los folders
Utilidades.folders_content(path)

- Contenido de la carpeta Test:
	- images/

- Contenido de la carpeta Train:
	- n01443537/
	- n01629819/
	- n01641577/
	- n01644900/
	- n01698640/
	- Más 195 carpetas.

- Contenido de la carpeta Val:
	- images/
	- val_annotations.txt

- Contenido de la carpeta Validation:
	- n01443537/
	- n01629819/
	- n01641577/
	- n01644900/
	- n01698640/
	- Más 195 carpetas.



**Conclusiones:**

- Después de un anális se llegó a la concusión de que la jerarquía de las carpetas es:
    - Test/
        - images/
            - Imagenes de Test
    - Train/
        - Carpeta con las imágenes de esa etiqueta/
            - images/
            - etiqueta_boxes.txt
    - Val/
        - images/
        - val_annotation.txt

## 3. Manejo de Carpetas
Necesitamos hacer un cambio en el orden de la carpeta de Validación (Val), ya que tiene un formato diferente que al de Entrenamiento (Train)

In [6]:
# Anotaciones del dataset de Validation
val_annotations = pd.read_csv('./Data/Val/val_annotations.txt', sep='\t', header=None)
val_annotations.columns = ['Img_Name', 'Clase', 'Xmin', 'Ymin', 'Xmax', 'Ymax'] # Cambiando columnas
val_annotations

,Img_Name,Clase,Xmin,Ymin,Xmax,Ymax
0,val_0.JPEG,n03444034,0,32,44,62
1,val_1.JPEG,n04067472,52,55,57,59
2,val_2.JPEG,n04070727,4,0,60,55
3,val_3.JPEG,n02808440,3,3,63,63
4,val_4.JPEG,n02808440,9,27,63,48
...,...,...,...,...,...,...
9995,val_9995.JPEG,n03085013,0,0,63,63
9996,val_9996.JPEG,n03444034,0,9,63,63
9997,val_9997.JPEG,n03424325,10,10,38,42
9998,val_9998.JPEG,n01629819,28,18,63,31


In [7]:
# Nombre de las imágenes por clase
clases_nombre = val_annotations.groupby('Clase')['Img_Name'].unique().reset_index()

# Llamando a la función que usamos para cambiar las imágenes de directorio
Utilidades.directory_name_change(df_nombres=clases_nombre, 
                                 df_data=val_annotations, 
                                 original_path='./Data/Val/images/', 
                                 new_path=r'./Data/Validation/')

## 4) Generadores de Datos

In [8]:
# Paths a las carpetas
train_path = ".\Data\Train"
val_path = ".\Data\Validation"

In [9]:
# Haciendo los Generadores de Datos
data_train = Utilidades.data_generator(path='./Data/Train/')
val_train = Utilidades.data_generator(path='./Data/Validation/')

Found 100000 images belonging to 200 classes.
Found 10000 images belonging to 200 classes.


# COSAS POR HACER
- Ver cómo poner el nombre de la clase y los bboxes